In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

# Part 1: Regression with l2 penalty and Akaike criterion

In [ ]:

import numpy as np


def generate_dataset(n, p, k, noise):
    """Q0.

    Build the following matrices with Gaussian entries (meaning that every entry is sampled
    from a standardized Gaussian distribution)
    - X of shape (n, p)
    - W of shape (p, k)
    - N of shape (n, k)
    Then form
    - Y = XW + noise * N
    and return W, X, Y
    """
    raise NotImplementedError


def f(Y, W, X, l):
    """Q1.

    Gives the value of the function
    f(Y, W, X, l) = 1/2 ||XW - Y||^2 + l/2 ||W||^2
    where || || is the frobenius norm

    Parameters
    ----------
    X : array of shape (n, p)
        Design matrix
    Y : array of shape (n, k)
    W : array of shape (p, k)
        Matrix of parameters
    l : float
        hyper-parameter

    Return
    --------
    value : float
        Value of f(Y, W, W, l)
    """
    raise NotImplementedError


def solver(X, Y, l=1):
    """Q2.

    Solves
    min_W 1/2||XW - Y||^2 + l/2 ||W||^2
    where || || is the frobenius norm

    Parameters
    ----------
    X : array of shape (n, p)
        Design matrix
    Y : array of shape (n, k)
        Target
    l : float
        hyper-parameter

    Return
    --------
    W: array of shape (p, k)
    W is such that
    1/2 ||XW - Y||^2 + l/2 ||W||^2 is minimized
    """
    raise NotImplementedError


def test_solver():
    n = 128
    p = 3
    k = 2
    W, X, Y = generate_dataset(n, p, k, 0)
    l = 1
    W_star = solver(X, Y, l)
    for _ in range(100):
        R = np.random.randn(p, k)
        assert f(Y, W_star + R, X, l) > f(Y, W_star, X, l)


test_solver()

In [ ]:
def cross_validated_score(X, Y, l):
    """Q3.

    Compute the cross validated score of the solver

    Divide the samples into 5 groups of equal size
    Leave out a group
    Take the samples corresponding to the groups that are not left out
    yielding arrays X_train of
    shape (n * 4/5, p) and Y_train of shape (n * 4/5, k)
    The remaining samples constitue X_test and Y_test.
    Then, apply the solver on X_train Y_train to get W_train
    and record
    the error of ||Y_test - X_test W_train ||^2

    Do this 5 times each time leaving a different group out
    and return the mean error.
    """
    raise NotImplementedError

In [ ]:
def plot_hyperparameter_curve(X, Y):
    """Q4.
    Plot cross_validated_score  in function of l
    with l in a grid of 100 points between 0 and 2
    """
    raise NotImplementedError


np.random.seed(0)
W, X, Y = generate_dataset(100, 20, 5, 1)
plot_hyperparameter_curve(X, Y)

In [ ]:


def akaike(Y, X, l, noise):
    """Q5.

    Compute the Akaike criterion for
    Y = X argmin_W f(Y, W, X, l)
    Recall that for the estimator A Y
    The akaike criterion is
    ||AY - Y ||^2 + 2 noise^2 tr(A) - noise^2 n

    Parameters
    ----------
    X : array of shape (n, p)
        Design matrix
    Y : array of shape (n, k)
    l : float
        hyper-parameter
    noise : float
        noise parameter

    Return
    -------
        akaike: float
    """
    raise NotImplementedError

In [ ]:


def optimal_akaike(Y, X, noise):
    """Q6.

    Compute the optimal parameter $l$ according to the Akaike estimator
    with l in a grid of 100 points between 0 and 2

    Parameters
    ----------
    X : array of shape (n, p)
        Design matrix
    Y : array of shape (n, k)

    Return
    --------
    l : float
        hyper-parameter
    """
    raise NotImplementedError


def test_akaike(Y, X, noise):
    """Q7.

    Compute the cross validated accuracy given by the akaike criterion


    Divide the samples into 5 groups of equal size
    Leave out a group
    Take the samples corresponding to the groups that are not left out
    yielding arrays X_train of
    shape (n * 4/5, p) and Y_train of shape (n * 4/5, k)
    The remaining samples constitue X_test and Y_test.
    Then, apply the solver on X_train and Y_train with l chosen
    according to the akaike criterion
    and record the error of ||Y_test - X_test W_train ||^2

    Do this 5 times each time leaving a different group out
    and return the mean error.
    """
    raise NotImplementedError


np.random.seed(0)
W, X, Y = generate_dataset(100, 20, 5, 1)
plot_hyperparameter_curve(X, Y)
print("Akaike cross validated score:", test_akaike(Y, X, 1))

# PART 2: FMRI
# What is fMRI ?

Functional magnetic resonance imaging (fMRI) is based on the fact that when local neural activity increases, increases in metabolism and blood flow lead to fluctuations of the relative concentrations of oxyhaemoglobin (the red cells in the blood that carry oxygen) and deoxyhaemoglobin (the same red cells after they have delivered the oxygen). Oxyhaemoglobin and deoxyhaemoglobin have different magnetic properties (diamagnetic and paramagnetic, respectively), and they affect the local magnetic field in different ways. The signal picked up by the MRI scanner is sensitive to these modifications of the local magnetic field. To record cerebral activity during functional sessions, the scanner is tuned to detect this “Blood Oxygen Level Dependent” (BOLD) signal.

Brain activity is measured in sessions that span several minutes, during which the participant performs some cognitive task and the scanner acquires brain images, typically every 2 or 3 seconds (the time between two successive image acquisition is called the Repetition time, or TR).

A cerebral MR image provides a 3D image of the brain that can be decomposed into voxels (the equivalent of pixels, but in 3 dimensions). The series of images acquired during a functional session provides, in each voxel, a time series of positive real number representing the MRI signal, sampled at the TR.

 One way to analyze times series consists in comparing them to a model built from our knowledge of the events that occurred during the functional session. Events can correspond to actions of the participant (e.g. button presses), presentations of sensory stimui (e.g. sound, images), or hypothesized internal processes (e.g. memorization of a stimulus), …

In [ ]:

# Prepare data and analysis parameters
# -------------------------------------
# Prepare the timing parameters.
t_r = 2.4
slice_time_ref = 0.5
# Prepare the volume-based fMRI data
from nilearn.datasets import fetch_localizer_first_level

data = fetch_localizer_first_level()
fmri_img = data.epi_img

# Prepare the experimental paradigm.
events_file = data.events
import pandas as pd

events = pd.read_table(events_file)

# Project the fMRI image to the surface
# -------------------------------------
#
# For this we need to get a mesh representing the geometry of the surface. We
# could use an individual mesh, but we first resort to a standard mesh, the
# so-called fsaverage5 template from the FreeSurfer software.

import nilearn

fsaverage = nilearn.datasets.fetch_surf_fsaverage()

# The projection function simply takes the fMRI data and the mesh.
# Note that those correspond spatially, are they are both in MNI space.
from nilearn import surface

texture = surface.vol_to_surf(fmri_img, fsaverage.pial_right)

In [ ]:
# We will perform first level analysis
# This involves computing the design matrix and fitting the model.
# We start by specifying the timing of fMRI frames.

import numpy as np

n_scans = texture.shape[1]
frame_times = t_r * (np.arange(n_scans) + 0.5)

###############################################################################
# Create the design matrix.
#
# We specify an hrf model containing the Glover model and its time derivative
# The drift model is implicitly a cosine basis with a period cutoff at 128s.
from nilearn.glm.first_level import make_first_level_design_matrix

design_matrix = make_first_level_design_matrix(
    frame_times, events=events, hrf_model="glover"
)

from nilearn import plotting

plotting.plot_design_matrix(design_matrix)
plotting.show()

In [ ]:
# The General linear model does a regression like operation
#

from nilearn.glm.first_level import run_glm

labels, estimates = run_glm(texture.T, design_matrix.values)

In [ ]:

# Estimate contrasts
# ------------------
# Specify the contrasts.
#
# For practical purpose, we first generate an identity matrix whose size is
# the number of columns of the design matrix.
contrast_matrix = np.eye(design_matrix.shape[1])

In [ ]:


###############################################################################
# At first, we create basic contrasts.
basic_contrasts = dict(
    [(column, contrast_matrix[i]) for i, column in enumerate(design_matrix.columns)]
)

###############################################################################
# Next, we add some intermediate contrasts and
# one contrast adding all conditions with some auditory parts.
basic_contrasts["audio"] = (
    basic_contrasts["audio_left_hand_button_press"]
    + basic_contrasts["audio_right_hand_button_press"]
    + basic_contrasts["audio_computation"]
    + basic_contrasts["sentence_listening"]
)

# one contrast adding all conditions involving instructions reading
basic_contrasts["visual"] = (
    basic_contrasts["visual_left_hand_button_press"]
    + basic_contrasts["visual_right_hand_button_press"]
    + basic_contrasts["visual_computation"]
    + basic_contrasts["sentence_reading"]
)

# one contrast adding all conditions involving computation
basic_contrasts["computation"] = (
    basic_contrasts["visual_computation"] + basic_contrasts["audio_computation"]
)

# one contrast adding all conditions involving sentences
basic_contrasts["sentences"] = (
    basic_contrasts["sentence_listening"] + basic_contrasts["sentence_reading"]
)

###############################################################################
# Finally, we create a more relevant contrast
#
# * 'audio - visual': probes the difference of activity between listening to some content or reading the same type of content (instructions, stories).
#
# Of course, we could define other contrasts, but we keep only this one for simplicity.


contrast_id = "left - right button press"
contrast_val = (
    basic_contrasts["audio_left_hand_button_press"]
    - basic_contrasts["audio_right_hand_button_press"]
    + basic_contrasts["visual_left_hand_button_press"]
    - basic_contrasts["visual_right_hand_button_press"]
)
# contrast_id = "audio-visual"
# contrast_val = basic_contrasts["audio"] - basic_contrasts["visual"]

from nilearn.glm.contrasts import compute_contrast
from nilearn import plotting

# compute contrast-related statistics
contrast = compute_contrast(labels, estimates, contrast_val, contrast_type="t")
# we present the associated z-score
p_value = contrast.p_value()
# we plot it on the surface, on the inflated fsaverage mesh,
# together with a suitable background to give an impression
# of the cortex folding.

In [ ]:
# Plot the relevant contrast
plotting.plot_surf_stat_map(
    fsaverage.infl_right,
    1 - p_value,
    hemi="right",
    title=contrast_id,
    colorbar=True,
    threshold=None,
    bg_map=fsaverage.sulc_right,
)
plotting.show()

In [ ]:


def plot_usual_threshold():
    """Q8.

    Threshold the contrast "left - right button press"
    so that only voxels
    with p-value lower than 5% are displayed

    Help: use the threshold argument of plot_surf_stat_map
    """
    raise NotImplementedError


plot_usual_threshold()

In [ ]:

def plot_bonferroni_correction():
    """Q9.

    Threshold the contrast "left - right button press"
    according to Bonferroni correction.
    """
    raise NotImplementedError


plot_bonferroni_correction()

In [ ]:

def plot_benjamini_yekutieli():
    """Q10.

    Threshold the contrast "left - right button press"
    according to Benjamini Yekutieli procedure
    """
    raise NotImplementedError


plot_benjamini_yekutieli()

In [ ]:

def plot_benjamini_hochberg():
    """Q11.

    Threshold the contrast "left - right button press"
    according to Benjamini Hochberg procedure
    """
    raise NotImplementedError


plot_benjamini_hochberg()